<a href="https://colab.research.google.com/github/L-Poca/Data_Pipeline/blob/rafael_cleaning/src/notebooks/transfer_learning_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Transfer Learning - COVID-19 Classification (Colab Pro)

**Objectif :** Classifier les radiographies pulmonaires avec Transfer Learning

**Modèles testés :**
- VGG16 (ImageNet)
- ResNet50 (ImageNet)
- EfficientNetB0 (ImageNet)
- InceptionV3 (ImageNet)

**Dataset :** COVID-19 Radiography (4 classes)
- COVID
- Normal
- Lung_Opacity
- Viral Pneumonia

**Stratégie :**
1. Feature extraction (freeze base)
2. Fine-tuning (unfreeze top layers)

## 📦 Cellule 1 : Configuration Standalone

**⚠️ REMPLACER CETTE CELLULE** par le contenu complet de `CELL_CONFIG_STANDALONE.py`

In [ ]:
"""
╔════════════════════════════════════════════════════════════════════════════╗
║  🎯 CELLULE DE CONFIGURATION STANDALONE - COPIER-COLLER DANS VOS NOTEBOOKS ║
╚════════════════════════════════════════════════════════════════════════════╝

INSTRUCTIONS:
-------------
1. Copiez TOUT le contenu de cette cellule
2. Collez-le comme PREMIÈRE CELLULE de votre notebook
3. Exécutez la cellule
4. Les variables sont prêtes à l'emploi !

Cette cellule est 100% autonome et fonctionne partout :
✅ Google Colab (clone + installe automatiquement)
✅ WSL / Linux Local
✅ Tout environnement Jupyter

APRÈS EXÉCUTION, VOUS POUVEZ UTILISER:
- config: Objet de configuration (config.batch_size, config.data_dir, etc.)
- ENV: Environnement détecté ('colab', 'wsl', 'local')
- Tous les imports des transformers

"""

# =============================================================================
# IMPORTS STANDARDS
# =============================================================================

import os
import sys
import subprocess
from pathlib import Path


# =============================================================================
# DÉTECTION AUTOMATIQUE DE L'ENVIRONNEMENT
# =============================================================================

def detect_environment():
    """Détecte l'environnement (colab, wsl, local)"""
    try:
        import google.colab
        return "colab"
    except ImportError:
        is_wsl = os.path.exists('/proc/version') and 'microsoft' in open('/proc/version').read().lower()
        return "wsl" if is_wsl else "local"

ENV = detect_environment()
print(f"🌍 Environnement: {ENV.upper()}")


# =============================================================================
# BOOTSTRAP COLAB (Clone + Install si nécessaire)
# =============================================================================

if ENV == "colab":
    print("\n🚀 Bootstrap Colab...")

    os.chdir('/content')
    if not os.path.exists('/content/Data_Pipeline'):
        print("📥 Clonage du repository...")
        subprocess.run(['git', 'clone', 'https://github.com/L-Poca/Data_Pipeline.git'], check=True)

    os.chdir('/content/Data_Pipeline')

    # Checkout de la branche rafael_cleaning
    result = subprocess.run(
        ['git', 'checkout', '-b', 'rafael_cleaning', 'origin/rafael_cleaning'],
        capture_output=True,
        text=True
    )
    if result.returncode != 0:
        # Si la branche locale existe déjà, juste switcher
        subprocess.run(['git', 'checkout', 'rafael_cleaning'], capture_output=True)

    print("📦 Installation des dépendances...")
    subprocess.run(['pip', 'install', '-r', 'requirements.txt', '--quiet'], check=True)

    print("📦 Installation du package...")
    result = subprocess.run(['pip', 'install', '-e', '.', '--quiet'], capture_output=True, text=True)
    if result.returncode != 0:
        print(f"⚠️ Erreur installation: {result.stderr}")
    else:
        print("✅ Package installé")

    print("💾 Montage Google Drive...")
    from google.colab import drive
    drive.mount('/content/drive')

    # Extraction dataset
    for archive in ['/content/drive/MyDrive/DS_COVID/archive_covid.zip']:
        if os.path.exists(archive):
            print("📦 Extraction dataset...")
            os.makedirs('./data/raw/', exist_ok=True)
            subprocess.run(['unzip', '-o', '-q', archive, '-d', './data/raw/COVID-19_Radiography_Dataset/'])
            break

    print("✅ Bootstrap terminé")


# =============================================================================
# CONFIGURATION DES CHEMINS
# =============================================================================

# Déterminer project_root selon l'environnement
if ENV == "colab":
    project_root = Path('/content/Data_Pipeline')
elif ENV == "wsl":
    project_root = Path('/home/cepa/DST/projet_DS/Data_Pipeline/Data_Pipeline')
else:  # local
    # Depuis un notebook dans src/notebooks/
    project_root = Path.cwd().parent.parent

# Ajouter src/ au sys.path pour les imports
# src_path = str(project_root / 'src')
# if src_path not in sys.path:
#     sys.path.insert(0, src_path)
#     print(f"✅ Chemin src/ ajouté: {src_path}")

# Charger la configuration depuis JSON
from src.utils.config import build_config

config = build_config(project_root, ENV)

# Exports pour compatibilité avec anciens notebooks
data_dir = config.data_dir
categories = config.classes
img_size = config.img_size


# =============================================================================
# IMPORTS DES TRANSFORMERS
# =============================================================================

try:
    from src.features.Pipelines.Transformateurs.image_loaders import ImageLoader
    from src.features.Pipelines.Transformateurs.image_preprocessing import (
        ImageResizer, ImageNormalizer, ImageFlattener
    )
    from src.features.Pipelines.Transformateurs.image_augmentation import (
        ImageAugmenter, ImageRandomCropper
    )
    from src.features.Pipelines.Transformateurs.image_features import (
        ImageHistogram, ImagePCA, ImageStandardScaler
    )
    print("✅ Transformers importés")
except ImportError as e:
    print(f"⚠️ Erreur import transformers: {e}")


# =============================================================================
# IMPORTS ML/DL
# =============================================================================

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

# =============================================================================
# CONFIGURATION MATPLOTLIB
# =============================================================================

plt.rcParams['figure.figsize'] = (15, 10)
sns.set_style('whitegrid')

# =============================================================================
# AFFICHAGE DU RÉSUMÉ
# =============================================================================

print("\n" + "=" * 70)
print("✅ CONFIGURATION PRÊTE - Data Pipeline")
print("=" * 70)
print(f"📂 Projet: {project_root}")
print(f"📊 Dataset: {data_dir}")
print(f"🏷️ Classes: {', '.join(categories)}")
print(f"🎛️ Images: {img_size}")
print(f"🔧 Batch: {config.batch_size} | Époques: {config.epochs}")
print(f"📐 Dataset accessible: {'✅' if data_dir.exists() else '❌'}")
if not data_dir.exists():
    print(f"   ⚠️ Créez le dossier ou placez les données dans: {data_dir}")
print("=" * 70)
print("\n💡 Variables disponibles:")
print("   • config: Configuration complète (Config object)")
print("   • project_root: Racine du projet (Path)")
print("   • data_dir: Dossier des données (Path)")
print("   • categories: Liste des 4 classes")
print("   • img_size: Taille des images (tuple)")
print("   • ENV: Environnement actuel")
print("\n🎯 Transformers disponibles:")
print("   • ImageLoader, ImageResizer, ImageNormalizer, ImageFlattener")
print("   • ImageAugmenter, ImageRandomCropper")
print("   • ImageHistogram, ImagePCA, ImageStandardScaler")
print("=" * 70)


## 1. Vérification GPU

In [ ]:
print("=" * 70)
print("VÉRIFICATION GPU")
print("=" * 70)

# Vérifier GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"\n✅ GPU disponible: {len(gpus)} GPU(s)")
    for gpu in gpus:
        print(f"   • {gpu.name}")

    # Configurer la mémoire GPU
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print("   • Memory growth activé")
else:
    print("\n⚠️ Pas de GPU disponible - utilisation CPU")
    print("   Pour activer GPU sur Colab: Runtime > Change runtime type > GPU")

print(f"\nTensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

## 2. Chargement des Données

In [ ]:
print("=" * 70)
print("CHARGEMENT DES DONNÉES")
print("=" * 70)

# Nombre d'images par classe (Colab Pro peut gérer plus)
N_IMAGES_PER_CLASS = 1000  # Ajuster selon GPU disponible

image_paths = []
labels = []
labels_int = []

for idx, cat in enumerate(categories):
    cat_path = data_dir / cat / 'images'
    if cat_path.exists():
        imgs = sorted(list(cat_path.glob('*.png')))[:N_IMAGES_PER_CLASS]
        image_paths.extend(imgs)
        labels.extend([cat] * len(imgs))
        labels_int.extend([idx] * len(imgs))
        print(f"  {cat:20s}: {len(imgs):4d} images")

labels_int = np.array(labels_int)

print(f"\n  Total: {len(image_paths)} images")
print(f"  Classes: {len(categories)}")
print(f"  Distribution: {np.bincount(labels_int)}")

## 3. Preprocessing Pipeline

In [ ]:
print("=" * 70)
print("PREPROCESSING PIPELINE")
print("=" * 70)

# Pipeline de préparation (RGB pour Transfer Learning)
prep_pipeline = Pipeline([
    ('load', ImageLoader(color_mode='RGB', verbose=False)),  # RGB pour ImageNet
    ('resize', ImageResizer(img_size=(224, 224), verbose=False)),  # 224x224 pour ImageNet
    ('norm', ImageNormalizer(method='minmax', verbose=False))
])

print("\nChargement des images...")
images = prep_pipeline.fit_transform(image_paths)

print(f"\n📊 Images préparées:")
print(f"  Shape: {images.shape}")
print(f"  Range: [{images.min():.3f}, {images.max():.3f}]")
print(f"  Dtype: {images.dtype}")

## 4. Train/Validation/Test Split

In [ ]:
from sklearn.model_selection import train_test_split

print("=" * 70)
print("TRAIN/VALIDATION/TEST SPLIT")
print("=" * 70)

# Split 70/15/15
X_train_val, X_test, y_train_val, y_test = train_test_split(
    images, labels_int,
    test_size=0.15,
    random_state=config.random_seed,
    stratify=labels_int
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val,
    test_size=0.176,  # 0.176 * 0.85 ≈ 0.15 du total
    random_state=config.random_seed,
    stratify=y_train_val
)

# One-hot encoding
y_train_cat = keras.utils.to_categorical(y_train, num_classes=len(categories))
y_val_cat = keras.utils.to_categorical(y_val, num_classes=len(categories))
y_test_cat = keras.utils.to_categorical(y_test, num_classes=len(categories))

print(f"\nTrain set: {X_train.shape[0]} images")
print(f"  Distribution: {np.bincount(y_train)}")
print(f"\nValidation set: {X_val.shape[0]} images")
print(f"  Distribution: {np.bincount(y_val)}")
print(f"\nTest set: {X_test.shape[0]} images")
print(f"  Distribution: {np.bincount(y_test)}")

## 5. Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("=" * 70)
print("DATA AUGMENTATION")
print("=" * 70)

# Data augmentation pour le training
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

# Pas d'augmentation pour validation/test
val_datagen = ImageDataGenerator()

print("\n✅ Data augmentation configurée")
print("   • Rotation: ±15°")
print("   • Shift: ±10%")
print("   • Flip horizontal: Oui")
print("   • Zoom: ±10%")
print("   • Brightness: 0.8-1.2")

## 6. Fonction de Création de Modèle Transfer Learning

In [ ]:
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB0, InceptionV3
from tensorflow.keras import layers, models

def create_transfer_model(base_model_name='VGG16', freeze_base=True, trainable_layers=0):
    """
    Crée un modèle de Transfer Learning

    Args:
        base_model_name: 'VGG16', 'ResNet50', 'EfficientNetB0', 'InceptionV3'
        freeze_base: Si True, freeze les poids du base model
        trainable_layers: Nombre de layers à rendre trainable (si freeze_base=False)
    """
    input_shape = (224, 224, 3)

    # Sélectionner le base model
    if base_model_name == 'VGG16':
        base_model = VGG16(
            weights='imagenet',
            include_top=False,
            input_shape=input_shape
        )
    elif base_model_name == 'ResNet50':
        base_model = ResNet50(
            weights='imagenet',
            include_top=False,
            input_shape=input_shape
        )
    elif base_model_name == 'EfficientNetB0':
        base_model = EfficientNetB0(
            weights='imagenet',
            include_top=False,
            input_shape=input_shape
        )
    elif base_model_name == 'InceptionV3':
        base_model = InceptionV3(
            weights='imagenet',
            include_top=False,
            input_shape=input_shape
        )
    else:
        raise ValueError(f"Base model inconnu: {base_model_name}")

    # Freeze/Unfreeze
    if freeze_base:
        base_model.trainable = False
    else:
        # Unfreeze les dernières couches
        for layer in base_model.layers[:-trainable_layers]:
            layer.trainable = False
        for layer in base_model.layers[-trainable_layers:]:
            layer.trainable = True

    # Construction du modèle
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(len(categories), activation='softmax')
    ])

    return model, base_model

print("✅ Fonction de création de modèle définie")

## 7. Phase 1 - Feature Extraction (VGG16)

In [ ]:
print("=" * 70)
print("PHASE 1: FEATURE EXTRACTION - VGG16")
print("=" * 70)

# Créer le modèle avec base freezé
model_vgg16, base_vgg16 = create_transfer_model('VGG16', freeze_base=True)

# Compilation
model_vgg16.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

print("\n📐 Architecture:")
model_vgg16.summary()

# Compter les paramètres
trainable_params = sum([tf.size(w).numpy() for w in model_vgg16.trainable_weights])
total_params = sum([tf.size(w).numpy() for w in model_vgg16.weights])
print(f"\nParamètres trainable: {trainable_params:,}")
print(f"Paramètres total: {total_params:,}")
print(f"Ratio trainable: {trainable_params/total_params:.1%}")

In [ ]:
# Callbacks
callbacks_vgg16 = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=config.early_stopping_patience,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=config.reduce_lr_factor,
        patience=config.reduce_lr_patience,
        min_lr=config.min_lr,
        verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        str(config.models_dir / 'vgg16_feature_extraction_best.keras'),
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# Entraînement
print("\n🚀 Entraînement Phase 1 (Feature Extraction)...\n")

history_vgg16_fe = model_vgg16.fit(
    train_datagen.flow(X_train, y_train_cat, batch_size=config.batch_size),
    validation_data=val_datagen.flow(X_val, y_val_cat, batch_size=config.batch_size),
    epochs=30,  # Moins d'époques pour feature extraction
    callbacks=callbacks_vgg16,
    verbose=1
)

print("\n✅ Phase 1 terminée")

## 8. Phase 2 - Fine-Tuning (VGG16)

In [ ]:
print("=" * 70)
print("PHASE 2: FINE-TUNING - VGG16")
print("=" * 70)

# Unfreeze les dernières 4 couches du base model
base_vgg16.trainable = True
for layer in base_vgg16.layers[:-4]:
    layer.trainable = False

print(f"\nLayers trainable: {sum([1 for l in base_vgg16.layers if l.trainable])}")
print(f"Layers frozen: {sum([1 for l in base_vgg16.layers if not l.trainable])}")

# Recompiler avec learning rate plus faible
model_vgg16.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),  # 10x plus faible
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

# Nouveaux callbacks
callbacks_vgg16_ft = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        str(config.models_dir / 'vgg16_finetuned_best.keras'),
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# Fine-tuning
print("\n🚀 Entraînement Phase 2 (Fine-Tuning)...\n")

history_vgg16_ft = model_vgg16.fit(
    train_datagen.flow(X_train, y_train_cat, batch_size=config.batch_size),
    validation_data=val_datagen.flow(X_val, y_val_cat, batch_size=config.batch_size),
    epochs=50,
    callbacks=callbacks_vgg16_ft,
    verbose=1
)

print("\n✅ Phase 2 terminée")

## 9. ResNet50 (Feature Extraction + Fine-Tuning)

In [ ]:
print("=" * 70)
print("RESNET50 - FEATURE EXTRACTION + FINE-TUNING")
print("=" * 70)

# Phase 1: Feature Extraction
model_resnet, base_resnet = create_transfer_model('ResNet50', freeze_base=True)

model_resnet.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

callbacks_resnet = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7),
    keras.callbacks.ModelCheckpoint(
        str(config.models_dir / 'resnet50_best.keras'),
        monitor='val_accuracy',
        save_best_only=True
    )
]

print("\n🚀 Phase 1: Feature Extraction\n")
history_resnet_fe = model_resnet.fit(
    train_datagen.flow(X_train, y_train_cat, batch_size=config.batch_size),
    validation_data=val_datagen.flow(X_val, y_val_cat, batch_size=config.batch_size),
    epochs=30,
    callbacks=callbacks_resnet,
    verbose=1
)

# Phase 2: Fine-Tuning
print("\n🚀 Phase 2: Fine-Tuning\n")
base_resnet.trainable = True
for layer in base_resnet.layers[:-10]:
    layer.trainable = False

model_resnet.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

history_resnet_ft = model_resnet.fit(
    train_datagen.flow(X_train, y_train_cat, batch_size=config.batch_size),
    validation_data=val_datagen.flow(X_val, y_val_cat, batch_size=config.batch_size),
    epochs=50,
    callbacks=callbacks_resnet,
    verbose=1
)

print("\n✅ ResNet50 terminé")

## 10. EfficientNetB0 (Feature Extraction + Fine-Tuning)

In [ ]:
print("=" * 70)
print("EFFICIENTNETB0 - FEATURE EXTRACTION + FINE-TUNING")
print("=" * 70)

# Phase 1: Feature Extraction
model_effnet, base_effnet = create_transfer_model('EfficientNetB0', freeze_base=True)

model_effnet.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

callbacks_effnet = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7),
    keras.callbacks.ModelCheckpoint(
        str(config.models_dir / 'efficientnetb0_best.keras'),
        monitor='val_accuracy',
        save_best_only=True
    )
]

print("\n🚀 Phase 1: Feature Extraction\n")
history_effnet_fe = model_effnet.fit(
    train_datagen.flow(X_train, y_train_cat, batch_size=config.batch_size),
    validation_data=val_datagen.flow(X_val, y_val_cat, batch_size=config.batch_size),
    epochs=30,
    callbacks=callbacks_effnet,
    verbose=1
)

# Phase 2: Fine-Tuning
print("\n🚀 Phase 2: Fine-Tuning\n")
base_effnet.trainable = True
for layer in base_effnet.layers[:-20]:
    layer.trainable = False

model_effnet.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

history_effnet_ft = model_effnet.fit(
    train_datagen.flow(X_train, y_train_cat, batch_size=config.batch_size),
    validation_data=val_datagen.flow(X_val, y_val_cat, batch_size=config.batch_size),
    epochs=50,
    callbacks=callbacks_effnet,
    verbose=1
)

print("\n✅ EfficientNetB0 terminé")

## 11. InceptionV3 (Feature Extraction + Fine-Tuning)

In [ ]:
print("=" * 70)
print("INCEPTIONV3 - FEATURE EXTRACTION + FINE-TUNING")
print("=" * 70)

# Phase 1: Feature Extraction
model_inception, base_inception = create_transfer_model('InceptionV3', freeze_base=True)

model_inception.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

callbacks_inception = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7),
    keras.callbacks.ModelCheckpoint(
        str(config.models_dir / 'inceptionv3_best.keras'),
        monitor='val_accuracy',
        save_best_only=True
    )
]

print("\n🚀 Phase 1: Feature Extraction\n")
history_inception_fe = model_inception.fit(
    train_datagen.flow(X_train, y_train_cat, batch_size=config.batch_size),
    validation_data=val_datagen.flow(X_val, y_val_cat, batch_size=config.batch_size),
    epochs=30,
    callbacks=callbacks_inception,
    verbose=1
)

# Phase 2: Fine-Tuning
print("\n🚀 Phase 2: Fine-Tuning\n")
base_inception.trainable = True
for layer in base_inception.layers[:-30]:  # InceptionV3 a beaucoup de layers
    layer.trainable = False

model_inception.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

history_inception_ft = model_inception.fit(
    train_datagen.flow(X_train, y_train_cat, batch_size=config.batch_size),
    validation_data=val_datagen.flow(X_val, y_val_cat, batch_size=config.batch_size),
    epochs=50,
    callbacks=callbacks_inception,
    verbose=1
)

print("\n✅ InceptionV3 terminé")

## 12. Évaluation sur Test Set

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("=" * 70)
print("ÉVALUATION SUR TEST SET")
print("=" * 70)

models_dict = {
    'VGG16': model_vgg16,
    'ResNet50': model_resnet,
    'EfficientNetB0': model_effnet,
    'InceptionV3': model_inception
}

results = {}

for name, model in models_dict.items():
    print(f"\n{'='*70}")
    print(f"{name}")
    print(f"{'='*70}")

    # Prédictions
    y_pred_proba = model.predict(X_test, verbose=0)
    y_pred = np.argmax(y_pred_proba, axis=1)

    # Métriques
    test_loss, test_acc, test_prec, test_rec = model.evaluate(X_test, y_test_cat, verbose=0)

    results[name] = {
        'accuracy': test_acc,
        'precision': test_prec,
        'recall': test_rec,
        'loss': test_loss,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }

    print(f"\n  Test Accuracy:  {test_acc:.4f}")
    print(f"  Test Precision: {test_prec:.4f}")
    print(f"  Test Recall:    {test_rec:.4f}")
    print(f"  Test Loss:      {test_loss:.4f}")

    # Rapport de classification
    print(f"\n  Classification Report:")
    print(classification_report(y_test, y_pred, target_names=categories, digits=4))

## 13. Comparaison des Modèles

In [ ]:
print("=" * 70)
print("COMPARAISON DES MODÈLES")
print("=" * 70)

# Tableau comparatif
print("\n{:<20s} {:>12s} {:>12s} {:>12s} {:>12s}".format(
    'Modèle', 'Accuracy', 'Precision', 'Recall', 'Loss'
))
print("-" * 70)

for name, res in sorted(results.items(), key=lambda x: x[1]['accuracy'], reverse=True):
    print("{:<20s} {:>12.4f} {:>12.4f} {:>12.4f} {:>12.4f}".format(
        name,
        res['accuracy'],
        res['precision'],
        res['recall'],
        res['loss']
    ))

# Meilleur modèle
best_model_name = max(results.items(), key=lambda x: x[1]['accuracy'])[0]
best_acc = results[best_model_name]['accuracy']

print("\n" + "=" * 70)
print(f"🏆 MEILLEUR MODÈLE: {best_model_name} (Accuracy: {best_acc:.4f})")
print("=" * 70)

## 14. Courbes d'Apprentissage

In [ ]:
fig, axes = plt.subplots(4, 2, figsize=(16, 22))

histories = {
    'VGG16': [history_vgg16_fe, history_vgg16_ft],
    'ResNet50': [history_resnet_fe, history_resnet_ft],
    'EfficientNetB0': [history_effnet_fe, history_effnet_ft],
    'InceptionV3': [history_inception_fe, history_inception_ft]
}

for idx, (name, hists) in enumerate(histories.items()):
    # Accuracy
    ax_acc = axes[idx, 0]
    for i, h in enumerate(hists):
        phase = "FE" if i == 0 else "FT"
        ax_acc.plot(h.history['accuracy'], label=f'{phase} Train', linewidth=2)
        ax_acc.plot(h.history['val_accuracy'], label=f'{phase} Val', linewidth=2, linestyle='--')

    ax_acc.set_xlabel('Epoch', fontsize=11)
    ax_acc.set_ylabel('Accuracy', fontsize=11)
    ax_acc.set_title(f'{name} - Accuracy', fontsize=12, weight='bold')
    ax_acc.legend(fontsize=9)
    ax_acc.grid(True, alpha=0.3)

    # Loss
    ax_loss = axes[idx, 1]
    for i, h in enumerate(hists):
        phase = "FE" if i == 0 else "FT"
        ax_loss.plot(h.history['loss'], label=f'{phase} Train', linewidth=2)
        ax_loss.plot(h.history['val_loss'], label=f'{phase} Val', linewidth=2, linestyle='--')

    ax_loss.set_xlabel('Epoch', fontsize=11)
    ax_loss.set_ylabel('Loss', fontsize=11)
    ax_loss.set_title(f'{name} - Loss', fontsize=12, weight='bold')
    ax_loss.legend(fontsize=9)
    ax_loss.grid(True, alpha=0.3)

plt.suptitle('Courbes d\'Apprentissage - Transfer Learning', fontsize=14, weight='bold', y=0.995)
plt.tight_layout()
plt.show()

## 15. Matrices de Confusion

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 14))
axes = axes.flatten()

for ax, (name, res) in zip(axes, results.items()):
    cm = confusion_matrix(y_test, res['y_pred'])

    sns.heatmap(
        cm, annot=True, fmt='d', cmap='Blues',
        xticklabels=categories, yticklabels=categories,
        ax=ax, cbar_kws={'label': 'Count'}
    )

    ax.set_title(f'{name}\nAcc: {res["accuracy"]:.3f}', fontsize=11, weight='bold')
    ax.set_xlabel('Predicted', fontsize=10)
    ax.set_ylabel('True', fontsize=10)

plt.suptitle('Matrices de Confusion - Test Set', fontsize=13, weight='bold')
plt.tight_layout()
plt.show()

## 16. Sauvegarde des Résultats

In [ ]:
import json
from datetime import datetime

# Créer un résumé
summary = {
    'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'dataset': {
        'total_images': len(image_paths),
        'train': len(X_train),
        'val': len(X_val),
        'test': len(X_test),
        'classes': categories
    },
    'models': {}
}

for name, res in results.items():
    summary['models'][name] = {
        'accuracy': float(res['accuracy']),
        'precision': float(res['precision']),
        'recall': float(res['recall']),
        'loss': float(res['loss'])
    }

summary['best_model'] = best_model_name

# Sauvegarder
results_file = config.results_dir / f'transfer_learning_results_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json'
with open(results_file, 'w') as f:
    json.dump(summary, f, indent=2)

print(f"✅ Résultats sauvegardés: {results_file}")
print(f"\n📊 Résumé:")
print(json.dumps(summary, indent=2))

## 17. Conclusion

### 📊 Résultats obtenus

Vous avez testé 4 architectures de Transfer Learning :
- **VGG16** : Feature extraction + Fine-tuning
- **ResNet50** : Feature extraction + Fine-tuning  
- **EfficientNetB0** : Feature extraction + Fine-tuning
- **InceptionV3** : Feature extraction + Fine-tuning

### 🎯 Prochaines étapes

1. **Interprétabilité** : GradCAM sur le meilleur modèle
2. **Ensemble Methods** : Combiner les 4 modèles (voting)
3. **Hyperparameter Tuning** : Tester différentes architectures de head
4. **More Data** : Augmenter le dataset si possible
5. **Cross-Validation** : Valider la robustesse des résultats

### 💾 Modèles sauvegardés

Les meilleurs modèles ont été sauvegardés dans `models/` :
- `vgg16_finetuned_best.keras`
- `resnet50_best.keras`
- `efficientnetb0_best.keras`
- `inceptionv3_best.keras`